In [6]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [2]:
import pandas as pd

dataset = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

pd.set_option("max_colwidth", 200)

dataset.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['SeniorCitizen'].mean()")

np.float64(0.1621468124378816)

In [4]:
df["MonthlyCharges"].mean()

np.float64(64.76169246059918)

In [13]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="deepseek-v3",
    model_provider="openai",
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

llm_with_tools = model.bind_tools([tool])

ValueError: Arg Precedence for the tool description value is as follows in docstring not found in function signature.

In [7]:
response = llm_with_tools.invoke(
    "我有一张表,名为'df'，请帮我计算MonthlyCharges字段的均值."
)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_22ac5e3234af4d47b1c828', 'function': {'arguments': '{"query":"df[\'MonthlyCharges\'].mean()"}', 'name': 'python_repl_ast'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 206, 'total_tokens': 235, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-89c563b6-4c87-909e-b6d2-043cbeb8da1d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9c9d4032-2efc-4528-b651-f126a41a05ee-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "df['MonthlyCharges'].mean()"}, 'id': 'call_22ac5e3234af4d47b1c828', 'type': 'tool_call'}], usage_metadata={'input_tokens': 206, 'output_tokens': 29, 'total_tokens': 235, 'input_token_details': {}, 'output_token_details': {}})

In [2]:
import requests
import json


def get_weather(loc):
    """ 查询即时天气函数
    :param loc: 必要参数，字符串类型，用户查询天气的具体城市名称，\
    注意，中国的城市需要用对应成是的英文名称代替，例如查询北京市额天气，则loc参数需要输入'Beijing'
    :return: OpenWeather API 查询即时天气的结果，具体URL请求地址为: https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析后的JSON格式对象，并用字符串的形式表示，其中包含了全部重要的天气信息
    """
    url = "https://api.openweathermap.org/data/2.5/weather"

    params = {
        "q": loc,
        "appid": os.getenv("OPENWEATHER_API_KEY"),
        "units": "metric",
        "lang": "zh_cn",
    }

    response = requests.get(url, params=params)

    data = response.json()

    return json.dumps(data)

In [7]:
get_weather("Beijing")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 802, "main": "Clouds", "description": "\\u591a\\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 33.94, "feels_like": 35.08, "temp_min": 33.94, "temp_max": 33.94, "pressure": 999, "humidity": 39, "sea_level": 999, "grnd_level": 993}, "visibility": 10000, "wind": {"speed": 2.56, "deg": 227, "gust": 2.38}, "clouds": {"all": 49}, "dt": 1752480268, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1752440238, "sunset": 1752493365}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [3]:
from langchain_core.tools import tool


@tool
def get_weather_tool(loc):
    """ 查询即时天气函数
    :param loc: 必要参数，字符串类型，用户查询天气的具体城市名称，\
    注意，中国的城市需要用对应成是的英文名称代替，例如查询北京市额天气，则loc参数需要输入'Beijing'
    :return: OpenWeather API 查询即时天气的结果，具体URL请求地址为: https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析后的JSON格式对象，并用字符串的形式表示，其中包含了全部重要的天气信息
    """
    return get_weather(loc)

In [9]:
print(get_weather_tool.name)
print(get_weather_tool.description)
print(get_weather_tool.args)

get_weather_tool
查询即时天气函数
   :param loc: 必要参数，字符串类型，用户查询天气的具体城市名称，    注意，中国的城市需要用对应成是的英文名称代替，例如查询北京市额天气，则loc参数需要输入'Beijing'
   :return: OpenWeather API 查询即时天气的结果，具体URL请求地址为: https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析后的JSON格式对象，并用字符串的形式表示，其中包含了全部重要的天气信息
{'loc': {'title': 'Loc'}}


In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="deepseek-v3",
    model_provider="openai",
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

tools = [get_weather_tool]

llm_with_tools = model.bind_tools(tools)

In [8]:
response = llm_with_tools.invoke("你好，请问成都的天气怎么样？")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_01ce6c7ecf13439baed8b6', 'function': {'arguments': '{"loc":"Chengdu"}', 'name': 'get_weather_tool'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 245, 'total_tokens': 268, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-faba23e5-676d-9638-8344-86c358df460b', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7adc58e2-6199-426f-8248-03b56dff2198-0', tool_calls=[{'name': 'get_weather_tool', 'args': {'loc': 'Chengdu'}, 'id': 'call_01ce6c7ecf13439baed8b6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 245, 'output_tokens': 23, 'total_tokens': 268, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
response.additional_kwargs

{'tool_calls': [{'id': 'call_01ce6c7ecf13439baed8b6',
   'function': {'arguments': '{"loc":"Chengdu"}', 'name': 'get_weather_tool'},
   'type': 'function',
   'index': 0}],
 'refusal': None}

In [10]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=get_weather_tool.name, first_tool_only=True)

llm_chain = llm_with_tools | parser

llm_chain.invoke("你好，请问成都天气怎么样？")

{'loc': 'Chengdu'}

In [11]:
get_weather_chain = llm_with_tools | parser | get_weather

get_weather_chain.invoke("你好，请问成都天气怎么样？")

'{"coord": {"lon": 7.511, "lat": 46.2932}, "weather": [{"id": 803, "main": "Clouds", "description": "\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 18.99, "feels_like": 19.4, "temp_min": 17.81, "temp_max": 19.23, "pressure": 1020, "humidity": 94, "sea_level": 1020, "grnd_level": 834}, "visibility": 10000, "wind": {"speed": 1.54, "deg": 230}, "clouds": {"all": 75}, "dt": 1752481120, "sys": {"type": 2, "id": 2007357, "country": "CH", "sunrise": 1752465103, "sunset": 1752520770}, "timezone": 7200, "id": 2658576, "name": "Loc", "cod": 200}'

In [25]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_prompt = PromptTemplate.from_template(
    """你将收到一段 JSON 格式的天气数据，请用简洁自然的方式将其转述给用户。
以下是天气 JSON 数据：

```json
{weather_json}
```

请将其转换为中文天气描述，例如：
“北京当前天气晴，气温为 23°C，湿度 58%，风速 2.1 米/秒。”
只返回一句话描述，不要其他说明或解释。"""
)

output_chain = output_prompt | model | StrOutputParser()

In [13]:
weather_json = '{"coord": {"lon": 7.511, "lat": 46.2932}, "weather": [{"id": 803, "main": "Clouds", "description": "\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 18.99, "feels_like": 19.4, "temp_min": 17.81, "temp_max": 19.23, "pressure": 1020, "humidity": 94, "sea_level": 1020, "grnd_level": 834}, "visibility": 10000, "wind": {"speed": 1.54, "deg": 230}, "clouds": {"all": 75}, "dt": 1752481120, "sys": {"type": 2, "id": 2007357, "country": "CH", "sunrise": 1752465103, "sunset": 1752520770}, "timezone": 7200, "id": 2658576, "name": "Loc", "cod": 200}'

result = output_chain.invoke({"weather_json": weather_json})
result

'洛当前天气多云，气温为18.99°C，湿度94%，风速1.54米/秒。'

In [26]:
full_chain = get_weather_chain | output_chain

full_chain.invoke("你好，请问成都天气怎么样？")

'洛当前天气多云，气温为 19°C，体感温度 19°C，湿度 92%，风速 0.51 米/秒。'

## 用 Agent 来使用 tool 更简单

In [27]:
from langchain.agents import create_tool_calling_agent, tool

tools = [get_weather_tool]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

model = init_chat_model(
    model="deepseek-v3",
    model_provider="openai",
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

agent = create_tool_calling_agent(model, tools, prompt)

In [28]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "请问成都今天的天气怎么样？"})
response



> Entering new AgentExecutor chain...

Invoking: `get_weather_tool` with `{'loc': 'Chengdu'}`


{"coord": {"lon": 104.0667, "lat": 30.6667}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 35.94, "feels_like": 42.54, "temp_min": 35.94, "temp_max": 35.94, "pressure": 1000, "humidity": 49, "sea_level": 1000, "grnd_level": 942}, "visibility": 10000, "wind": {"speed": 3, "deg": 0}, "clouds": {"all": 40}, "dt": 1752483158, "sys": {"type": 1, "id": 9674, "country": "CN", "sunrise": 1752444629, "sunset": 1752494894}, "timezone": 28800, "id": 1815286, "name": "Chengdu", "cod": 200}成都今天的天气是多云，当前温度为35.94℃，体感温度为42.54℃。湿度为49%，气压为1000 hPa。风速为3 m/s，风向为0度（北风）。能见度为10公里。今天的日出时间为早上7点10分，日落时间为晚上7点48分。

> Finished chain.


{'input': '请问成都今天的天气怎么样？',
 'output': '成都今天的天气是多云，当前温度为35.94℃，体感温度为42.54℃。湿度为49%，气压为1000 hPa。风速为3 m/s，风向为0度（北风）。能见度为10公里。今天的日出时间为早上7点10分，日落时间为晚上7点48分。'}

In [29]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {"input": "请问北京和杭州今天的天气怎么样？哪个城市更热？"}
)
response



> Entering new AgentExecutor chain...

Invoking: `get_weather_tool` with `{'loc': 'Beijing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 33.94, "feels_like": 35.58, "temp_min": 33.94, "temp_max": 33.94, "pressure": 999, "humidity": 41, "sea_level": 999, "grnd_level": 992}, "visibility": 10000, "wind": {"speed": 2.89, "deg": 229, "gust": 1.75}, "clouds": {"all": 50}, "dt": 1752483900, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1752440238, "sunset": 1752493365}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}
Invoking: `get_weather_tool` with `{'loc': 'Hangzhou'}`


{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 803, "main": "Clouds", "description": "\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 34.95, "feels_like": 41.95, "temp_min": 34.95, "temp_max": 34.95, "pressure": 999, "humidit

{'input': '请问北京和杭州今天的天气怎么样？哪个城市更热？',
 'output': '北京的天气是多云，当前温度为33.94°C，体感温度35.58°C，湿度为41%。  \n\n杭州的天气也是多云，当前温度为34.95°C，体感温度41.95°C，湿度为64%。  \n\n从当前温度来看，**杭州比北京更热**，而且杭州的体感温度更高，湿度也较大。'}

In [31]:
@tool
def write_file(content):
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return: 是否成功写入
    """
    return "已成功写入本地文件"

In [32]:
tools = [get_weather_tool, write_file]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是天气助手，请根据用户的问题，给出相应的天气信息，如果用户需要将查询结果写入文件，请使用write_file工具",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

model = init_chat_model(
    model="deepseek-v3",
    model_provider="openai",
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

agent = create_tool_calling_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "请问北京和杭州现在的天气？并写入文件"})
response



> Entering new AgentExecutor chain...

Invoking: `get_weather_tool` with `{'loc': 'Beijing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 33.94, "feels_like": 35.58, "temp_min": 33.94, "temp_max": 33.94, "pressure": 999, "humidity": 41, "sea_level": 999, "grnd_level": 992}, "visibility": 10000, "wind": {"speed": 2.89, "deg": 229, "gust": 1.75}, "clouds": {"all": 50}, "dt": 1752483900, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1752440238, "sunset": 1752493365}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}
Invoking: `get_weather_tool` with `{'loc': 'Hangzhou'}`


{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 803, "main": "Clouds", "description": "\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 34.95, "feels_like": 41.95, "temp_min": 34.95, "temp_max": 34.95, "pressure": 999, "humidit

{'input': '请问北京和杭州现在的天气？并写入文件',
 'output': '北京现在的天气：多云，温度33.94°C，体感温度35.58°C，湿度41%，风速2.89m/s。  \n杭州现在的天气：多云，温度34.95°C，体感温度41.95°C，湿度64%，风速3.82m/s。  \n\n天气信息已成功写入文件。'}